In [1]:
import ast
import copy
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from aocd import get_data, submit

DAY = 22
YEAR = 2022

In [2]:
# use test data
raw_test = """        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5"""

# use real data
raw = get_data(day=DAY, year=YEAR)

print(raw_test)

        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5


In [3]:
def parse_data(data):
    grid, inst = data.split("\n\n")

    steps = list(map(int, re.findall("[0-9]+", inst)))
    dirs = [None] + re.findall("(R|L)", inst)
    inst = [(s, d) for d, s in zip(dirs, steps)]

    coords = {}
    grid = grid.split("\n")
    for rdx, row in enumerate(grid):
        for cdx, el in enumerate(row):
            if el == " ":
                continue
            coords[(cdx + 1, rdx + 1)] = 0 if el == "#" else 1

    return coords, inst


dummy = parse_data(raw_test)
real = parse_data(raw)

dummy

({(9, 1): 1,
  (10, 1): 1,
  (11, 1): 1,
  (12, 1): 0,
  (9, 2): 1,
  (10, 2): 0,
  (11, 2): 1,
  (12, 2): 1,
  (9, 3): 0,
  (10, 3): 1,
  (11, 3): 1,
  (12, 3): 1,
  (9, 4): 1,
  (10, 4): 1,
  (11, 4): 1,
  (12, 4): 1,
  (1, 5): 1,
  (2, 5): 1,
  (3, 5): 1,
  (4, 5): 0,
  (5, 5): 1,
  (6, 5): 1,
  (7, 5): 1,
  (8, 5): 1,
  (9, 5): 1,
  (10, 5): 1,
  (11, 5): 1,
  (12, 5): 0,
  (1, 6): 1,
  (2, 6): 1,
  (3, 6): 1,
  (4, 6): 1,
  (5, 6): 1,
  (6, 6): 1,
  (7, 6): 1,
  (8, 6): 1,
  (9, 6): 0,
  (10, 6): 1,
  (11, 6): 1,
  (12, 6): 1,
  (1, 7): 1,
  (2, 7): 1,
  (3, 7): 0,
  (4, 7): 1,
  (5, 7): 1,
  (6, 7): 1,
  (7, 7): 1,
  (8, 7): 0,
  (9, 7): 1,
  (10, 7): 1,
  (11, 7): 1,
  (12, 7): 1,
  (1, 8): 1,
  (2, 8): 1,
  (3, 8): 1,
  (4, 8): 1,
  (5, 8): 1,
  (6, 8): 1,
  (7, 8): 1,
  (8, 8): 1,
  (9, 8): 1,
  (10, 8): 1,
  (11, 8): 0,
  (12, 8): 1,
  (9, 9): 1,
  (10, 9): 1,
  (11, 9): 1,
  (12, 9): 0,
  (13, 9): 1,
  (14, 9): 1,
  (15, 9): 1,
  (16, 9): 1,
  (9, 10): 1,
  (10, 10): 1,
  (1

# Part 1

In [4]:
def score(current):
    col, row, fac = current
    return 1000 * row + 4 * col + fac


def move(cave, current, inst):
    cx, cy, cd = current
    nx, ny, nd = current
    steps, rotation = inst

    if rotation is not None:
        nd = (cd + 1) % 4 if rotation == "R" else (nd - 1) % 4

    if nd in [0, 2]:  # right or left
        sign = -1 if nd == 2 else 1
        x_coords = [x for (x, y), el in cave.items() if y == cy]
        x_min, x_max = min(x_coords), max(x_coords)
        width = x_max - x_min + 1

        for s in range(1, steps + 1):
            new_x = (cx + s * sign - x_min) % width + x_min
            if cave[(new_x, cy)] == 0:
                break
            nx = new_x

    else:
        sign = -1 if nd == 3 else 1
        y_coords = [y for (x, y), el in cave.items() if x == cx]
        y_min, y_max = min(y_coords), max(y_coords)
        height = y_max - y_min + 1

        for s in range(1, steps + 1):
            new_y = (cy + s * sign - y_min) % height + y_min
            if cave[(cx, new_y)] == 0:
                break
            ny = new_y

    return (nx, ny, nd)

In [5]:
grid, inst = copy.deepcopy(real)

sx = min([x for (x, y), el in grid.items() if y == 1 and el == 1])
sy = 1
sd = 0
current = (sx, sy, sd)

for cmd in inst:
    current = move(grid, current, cmd)

result = score(current)
result

50412

In [6]:
# submit(result, part="a", day=DAY, year=YEAR)

# Part 2

In [7]:
def get_next(grid, current, check=True):
    cx, cy, cd = current
    nx, ny, nd = current
    a = int((len(grid.keys()) / 6) ** 0.5)

    if cd in [0, 2]:
        x_coords = [x for (x, y), el in grid.items() if y == cy]
        x_min, x_max = min(x_coords), max(x_coords)
        sign = -1 if cd == 2 else 1

        tx = cx + sign
        if x_min <= tx <= x_max:
            if check and grid[(tx, cy)] == 0:
                return (cx, cy, cd), True
            return (tx, cy, cd), False

        else:
            if (cy - 1) // a == 0:
                tx, ty, td = cx - a, 3 * a - (cy - 1) % a, 2 if cd == 0 else 0
            elif (cy - 1) // a == 1:
                tx, ty, td = cy + sign * a, cx - sign * a, 3 if cd == 0 else 1
            elif (cy - 1) // a == 2:
                tx, ty, td = cx + a, a - (cy - 1) % a, 2 if cd == 0 else 0
            else:
                if tx > x_max:
                    tx, ty, td = cy - 2 * a, cx + 2 * a, 3
                else:
                    tx, ty, td = cy - 2 * a, cx, 1
    else:
        y_coords = [y for (x, y), el in grid.items() if x == cx]
        y_min, y_max = min(y_coords), max(y_coords)
        sign = -1 if cd == 3 else 1

        ty = cy + sign
        if y_min <= ty <= y_max:
            if check and grid[(cx, ty)] == 0:
                return (cx, cy, cd), True
            return (cx, ty, cd), False
        else:
            if (cx - 1) // a == 0:
                if ty > y_max:
                    tx, ty, td = cx + 2 * a, 1, 1
                else:
                    tx, ty, td = cy - a, cx + a, 0
            elif (cx - 1) // a == 1:
                if ty > y_max:
                    tx, ty, td = cy - 2 * a, cx + 2 * a, 2
                else:
                    tx, ty, td = cy, cx + 2 * a, 0
            else:
                if ty > y_max:
                    tx, ty, td = cy + a, cx - a, 2
                else:
                    tx, ty, td = cx - 2 * a, 4 * a, 3

    if check and grid[(tx, ty)] == 0:
        return (cx, cy, cd), True
    return (tx, ty, td), False


def move_v2(grid, current, inst):
    cx, cy, cd = current
    steps, rotation = inst

    if rotation is not None:
        cd = (cd + 1) % 4 if rotation == "R" else (cd - 1) % 4

    nxt = (cx, cy, cd)
    has_hit_wall = False
    while not has_hit_wall and steps > 0:
        nxt, has_hit_wall = get_next(grid, nxt)
        steps -= 1

    return nxt


# F -> B
assert get_next(grid, (1, 200, 1), False)[0] == (101, 1, 1)
assert get_next(grid, (50, 200, 1), False)[0] == (150, 1, 1)

# D -> C
assert get_next(grid, (1, 101, 3), False)[0] == (51, 51, 0)
assert get_next(grid, (50, 101, 3), False)[0] == (51, 100, 0)

# E -> F
assert get_next(grid, (51, 150, 1), False)[0] == (50, 151, 2)
assert get_next(grid, (100, 150, 1), False)[0] == (50, 200, 2)

# A -> F
assert get_next(grid, (51, 1, 3), False)[0] == (1, 151, 0)
assert get_next(grid, (100, 1, 3), False)[0] == (1, 200, 0)

# B -> C
assert get_next(grid, (101, 50, 1), False)[0] == (100, 51, 2)
assert get_next(grid, (150, 50, 1), False)[0] == (100, 100, 2)

# B -> F
assert get_next(grid, (101, 1, 3), False)[0] == (1, 200, 3)
assert get_next(grid, (150, 1, 3), False)[0] == (50, 200, 3)

In [8]:
grid, inst = copy.deepcopy(real)

sx = min([x for (x, y), el in grid.items() if y == 1 and el == 1])
sy = 1
sd = 0
current = (sx, sy, sd)

print(current)
print()
for idx, cmd in enumerate(inst):
    current = move_v2(grid, current, cmd)
    # print(idx, cmd, current)

result = score(current)
result

(51, 1, 0)



130068

In [9]:
# submit(result, part="b", day=DAY, year=YEAR)